In [131]:
import boto3
import json
import os
from functools import reduce
import pprint
from prettytable import PrettyTable
from youtube_transcript_api import YouTubeTranscriptApi
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from google.oauth2.credentials import Credentials

from phrasefinder import phrasefinder as pf
from sklearn.feature_extraction import stop_words
import time
from tqdm import tqdm
import statistics
from functools import reduce

comprehend = boto3.client(service_name='comprehend', region_name='us-east-1')

def getBestVideoList(keywords):
    scopes = ["https://www.googleapis.com/auth/youtube.force-ssl"]
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "0"

    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = "./google_credentials/client_secret_754636752811-rmth1g8e3dl144jda8fddh1ihhj413um.apps.googleusercontent.com.json"

    # Get credentials and create an API client
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes)

    credentials =Credentials(
        None,
        refresh_token="1//0fNppFYz3o7ABCgYIARAAGA8SNwF-L9IrgIZJAKCn9iSH_172SxyT6cA3mMHDlSQ0MTj9MmKTc6zZRnSy1nwMW5kRkl52JYb4jhg",
        token_uri="https://accounts.google.com/o/oauth2/token",
        client_id="754636752811-rmth1g8e3dl144jda8fddh1ihhj413um.apps.googleusercontent.com",
        client_secret="KhUufHmhS8XI0srgpP__cTCr"
    )

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, credentials=credentials)

    request = youtube.search().list(
        part="snippet",
        maxResults=30,
        q=keywords,
        relevanceLanguage='en'
    )
    response = request.execute()
    return response['items']

"""     best_content_list = []
    with os.scandir('./data') as entries:
        for entry in entries:
            with open('./data/' + entry.name, 'r') as file:
                best_content_list.append(file.read())
    return best_content_list """

def getKeyPhrases(content_list):
    def insertFrequency(keyPhrase):
        frequency = {'overall': 0, 'min': 0, 'max': 0, 'frequency_by_content': []}
        for content in content_list:
            _freq = content.count(keyPhrase['Text'])
            frequency['overall'] += _freq
            frequency['min'] = _freq if frequency['min'] == 0 else min(frequency['min'], _freq)
            frequency['max'] = max(frequency['max'], _freq)
            frequency['frequency_by_content'].append(_freq)

        frequency['doc_freq'] = 0
        for e in frequency['frequency_by_content']:
            frequency['doc_freq'] += e>0

        frequency['median'] = statistics.median(frequency['frequency_by_content'])
        return {**keyPhrase, 'frequency': frequency}

    def insertNgramFrequency(keyPhrase):
         # Set up your query.
        query = keyPhrase['Text']

        # Optional: set the maximum number of phrases to return.
        options = pf.SearchOptions()
        options.topk = 1

        # Send the request.
        try:
            result = pf.search(pf.Corpus.AMERICAN_ENGLISH, query, options)
            if result.error:
                print('Request for {} was not successful: {}'.format(query, result.error['message']))
                return
            # print(result.phrases[0].volume_count)
            time.sleep(0.02)
            return {**keyPhrase, 'ngram_volume_count': result.phrases[0].volume_count, 'ngram_match_count': result.phrases[0].match_count}
        except Exception as error:
            print('Fatal error for {}: {}'.format(query, error))
            return {**keyPhrase, 'ngram_volume_count': 0, 'ngram_match_count': 0}
        
        

    def filterPhrases(phrasesByContent):
        phrases = reduce(lambda x,y: x+y, phrasesByContent)
        phrases = list(filter(lambda x: x['Score'] >= 0.999, phrases))

        # trim stop words from begining and end
        def stripStopWords(phrase):
            for w in stop_words.ENGLISH_STOP_WORDS:
                if phrase['Text'].startswith(w+' '):
                    phrase['Text'] = phrase['Text'][len(w)+1:]
                    break
                elif phrase['Text'].endswith(' '+w):
                    phrase['Text'] = phrase['Text'][:-len(w)+1]
            return phrase
        phrases = list(map(stripStopWords, phrases))

        # only keep bigrams
        phrases = list(filter(lambda x: len(x['Text'].split(' ')) >= 1, phrases))

        # remove duplicates
        unique_phrases=[]
        phrase_set = set()
        for phrase in phrases:
            if phrase['Text'] not in phrase_set:
                phrase_set.add(phrase['Text'])
                unique_phrases.append(phrase)

        print('# Unique phrases: {}'.format(len(unique_phrases)))

        # add frequency for each phrase
        unique_phrases = list(map(insertFrequency, unique_phrases))

        # keep phrases showing up at least once in all documents
        unique_phrases = list(filter(lambda x: x['frequency']['min'] >= 0, unique_phrases))

        # phrase should have at least 3 chars
        unique_phrases = list(filter(lambda x: len(x['Text']) >= 3, unique_phrases))

        # phrase should not have weird symbols
        unique_phrases = list(filter(lambda x: all(x.isalpha() or x.isspace() or x == '-' or x =='$' or x.isdigit() for x in x['Text']), unique_phrases))

        print('# phrases: {}'.format(len(unique_phrases)))

        # add ngrams frequency for each phrase
        for i in tqdm(range(len(unique_phrases))):
            unique_phrases[i] = insertNgramFrequency(unique_phrases[i])
        #unique_phrases = list(map(insertNgramFrequency, unique_phrases))

        phrases = unique_phrases

        # sort key phrases by overall frequency
        phrases.sort(reverse=True, key=lambda x: x['frequency']['overall'])

        print(len(phrases))

        # return most frequent top phrases
        return phrases

    # comprehend api requires each content to be less than 5000 bytes
    # see https://docs.aws.amazon.com/comprehend/latest/dg/guidelines-and-limits.html
    content_list_splited = []
    for content in content_list:
        content_list_splited.extend([content[i: i+4000] for i in range(0, len(content), 4000)])
    """ for content in content_list_splited:
        print(len(content))
 """
    def chunks(l, n):
        for i in range(0, len(l), n):
         yield l[i:i + n]

    content_list_splited_chunks = list(chunks(content_list_splited, 25))

    keyPhrasesByContent = []
    entitiesByContent = []
    for chunk in content_list_splited_chunks:
        keyPhrasesResponse = comprehend.batch_detect_key_phrases(TextList=chunk, LanguageCode='en')
        entitiesResponse = comprehend.batch_detect_entities(TextList=chunk, LanguageCode='en')

        keyPhrasesByContent.extend(list(map(lambda x: x['KeyPhrases'], keyPhrasesResponse['ResultList'])))
        entitiesByContent.extend(list(map(lambda x: x['Entities'], entitiesResponse['ResultList'])))

    # return filterPhrases(keyPhrasesByContent), filterPhrases(entitiesByContent)
    return filterPhrases(keyPhrasesByContent)

def getTranscript(video_id):
    print('Getting transcript for '+video_id)
    return YouTubeTranscriptApi.get_transcript(video_id)

In [82]:
keywords = 'How To Buy Your First Rental'
print('Working on keywords: ' + keywords)
video_list = getBestVideoList(keywords)
# video_list = [{'id': {'videoId': 'kqMtDrsc5Pw'}}]
transcript_list = []
for video in video_list:
    try: 
        transcript_list.append(getTranscript(video['id']['videoId']))
    except:
        pass

content_list = []
for transcript in transcript_list:
    content = '. '.join(list(map(lambda x: x['text'], transcript)))
    # print(content)
    # print("\n\n")
    content_list.append(content)

Working on keywords: How To Buy Your First Rental
Getting transcript for bJx7_1rWC6U
Getting transcript for 7TB_eRhSNV4
Getting transcript for u83O2l1QEj4
Getting transcript for nb9gtT-BqRc
Getting transcript for IOqlt3dwb_c
Getting transcript for LxAniFgeCOg
Getting transcript for qiKg3790dtA
Getting transcript for zoCGqNkBsbA
Getting transcript for a5RExfeU4UY
Getting transcript for yv4ZIaiRMZM
Getting transcript for CPJp9Ch37UM
Getting transcript for zV7ESWTdKQ0
Getting transcript for ShOrTMtsxuc
Getting transcript for vMZSFGpAtcM
Getting transcript for V_jHm7Dtdmg
Getting transcript for aGl3glZN74o
Getting transcript for eH5lvuxvdCk
Getting transcript for rFN_VOQjVqE
Getting transcript for _HX22c0YD2s
Getting transcript for E-WipmObnUM
Getting transcript for M-kMPNa7Lo4
Getting transcript for NCACgUcHJL0
Getting transcript for HqGawxcJ4NU
Getting transcript for _D5Z_qt-DkA
Getting transcript for QzB43V74V54
Getting transcript for pQuUdXB_IaU
Getting transcript for fWqOdY9c3TM
Getti

In [132]:
# keyPhrases, entities = getKeyPhrases(content_list)
keyPhrases = getKeyPhrases(content_list)

# Unique phrases: 2318
 38%|███▊      | 841/2230 [06:02<09:58,  2.32it/s]Fatal error for really
big paydays: list index out of range
 40%|████      | 898/2230 [06:27<09:30,  2.33it/s]Fatal error for very first rental
property: list index out of range
 58%|█████▊    | 1287/2230 [09:28<07:26,  2.11it/s]Fatal error for the 
new way: list index out of range
 58%|█████▊    | 1290/2230 [09:30<07:32,  2.08it/s]Fatal error for the 
difference: list index out of range
 58%|█████▊    | 1299/2230 [09:35<07:31,  2.06it/s]Fatal error for real 
estate coach: list index out of range
 58%|█████▊    | 1301/2230 [09:35<07:16,  2.13it/s]Fatal error for these 
people: list index out of range
 58%|█████▊    | 1302/2230 [09:36<07:10,  2.15it/s]Fatal error for that 
contract: list index out of range
 59%|█████▉    | 1312/2230 [09:41<07:10,  2.13it/s]Fatal error for rental 
properties: list index out of range
 59%|█████▉    | 1319/2230 [09:44<07:26,  2.04it/s]Fatal error for that 
terrible process: list index

In [136]:
import math

total_words = 0
def insertTfidf(phrase):
    # 1.4 M books
    phrase['tfidf'] = (phrase['frequency']['overall'] / total_words) * (1+math.log(1300000/(1+phrase['ngram_volume_count'])))
    return phrase

def printTable(phrases):
    table = PrettyTable()
    table.field_names=['phrase', 'score', 'overall_freq', 'min_freq', 'max_freq', 'median_freq', 'doc_freq', 'ngram_volume_count', 'ngram_match_count', 'tfidf', 'entity_type']
    for phrase in phrases:
        entity_type = 'N/A'
        if 'Type' in phrase:
            entity_type = phrase['Type']
        table.add_row([phrase['Text'], phrase['Score'], phrase['frequency']['overall'], phrase['frequency']['min'], phrase['frequency']['max'], phrase['frequency']['median'], phrase['frequency']['doc_freq'], phrase['ngram_volume_count'], phrase['ngram_match_count'], phrase['tfidf'], entity_type])
    print(table)

for content in content_list:
    total_words += len(content.split())

print(len(keyPhrases))
keyPhrases = list(map(insertTfidf, keyPhrases))
# print top keyphrases table
keyPhrases.sort(reverse=True, key=lambda x: x['tfidf'])
printTable(keyPhrases)
#printTable(entities)

2230
+-------------------------------------+--------------------+--------------+----------+----------+-------------+----------+--------------------+-------------------+------------------------+-------------+
|                phrase               |       score        | overall_freq | min_freq | max_freq | median_freq | doc_freq | ngram_volume_count | ngram_match_count |         tfidf          | entity_type |
+-------------------------------------+--------------------+--------------+----------+----------+-------------+----------+--------------------+-------------------+------------------------+-------------+
|           rental property           | 0.9999733567237854 |     142      |    1     |    26    |      3      |    22    |       23333        |       84375       |  0.008126740372152106  |     N/A     |
|                rental               | 0.9999950528144836 |     312      |    1     |    44    |      7      |    29    |       435103       |      2514586      |  0.0074498654985649